# Sec-Gemini Python SDK demo notebook

This notebook demonstrates how to use the Sec-Gemini Python SDK to interact with the Sec-Gemini API.

In [4]:
%load_ext autoreload
%autoreload 2

from secgemini import SecGemini

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## initialization

the API key is usually loaded for the SG_API_KEY environment variable but you can pass it directly to the client

```python
from sec_gemini import SecGemini
client = SecGemini(api_key="your_api_key")
```


In [5]:
# sg = SecGemini()

# alternative way to test local server
# BASE_URL = 'http://localhost:8000'
# WSS_URL = 'ws://localhost:8000'
# sg = SecGemini(base_url=BASE_URL, base_websockets_url=WSS_URL)

# normal way to connect to the main server
sg = SecGemini()
sg.display_info()   # show current user info

                    User Information                    
╭──────────────────────┬───────────────────────────────╮
│ Attribute            │ Value                         │
├──────────────────────┼───────────────────────────────┤
│ Type                 │ user                          │
│ User ID              │ dev-neverlog@secgemini.google │
│ Organization ID      │ google:p9                     │
│ Never log?           │ True                          │
│ Key Expiration Time  │ Never                         │
│ Can disable session  │ False                         │
│ logging?             │                               │
│ Cam use experimental │ False                         │
│ features?            │                               │
│ TPM Quota            │ 400000                        │
│ RPM Quota            │ 100                           │
╰──────────────────────┴───────────────────────────────╯

                                                                                                          sec-gemini-1.1-stable                                                                                                           
╭───────────────┬────────────┬─────────┬──────────┬───────────┬───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Agent         │ Vendor     │ Version │ Enabled? │ Optional? │ Experimental? │ Description                                                                                                                                              │
├───────────────┼────────────┼─────────┼──────────┼───────────┼───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ orchestrator  │ Sec-Gemini │ 1       │ True     │ False     │ False         │ Sec-Gemini main orchestrator.                                                                                                                            │
│ network_agent │ Sec-Gemini │ 1       │ True     │ True      │ False         │ You are an agent specialized in network security including network diagonistics, network configuration, network forensics, and network traffic analysis. │
╰───────────────┴────────────┴─────────┴──────────┴───────────┴───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                                                   Sessions                                                                   
╭──────────────────────────────────┬─────────────┬─────────────────┬───────┬────────┬──────────────────────┬──────────────────────┬──────────╮
│                                  │             │                 │       │        │                      │                      │ TTL      │
│ ID / Name                        │ Description │ State           │ #Msg  │ #Files │ Created              │ Updated              │ (sec)    │
├──────────────────────────────────┼─────────────┼─────────────────┼───────┼────────┼──────────────────────┼──────────────────────┼──────────┤
│ d68f816474984c4284b87345255705c3 │             │ start           │ 1     │ 0      │ 2025-05-04 01:33:42  │ 2025-05-04 01:33:42  │ 259200   │
│ young-dsa                        │             │                 │       │        │                      │                      │          │
╰──────────────────────────────────┴─────────────┴─────────────────┴───────┴────────┴──────────────────────┴──────────────────────┴──────────╯

## Basic Usage

The basic worflow involves:
1. **Create a client**: This is the main entry point to the SDK. It handles authentication and provides methods to interact with the API as demonstrated above.

2. **Create a session**: Having resumable sessions allows to have multiples workflows running in parallel that can be resumed on a different machine. Each session has its own state and can be used to send and receive messages independently.

3. **Query the model**: The model can be queried using the `query` method. **Warning** This is the simplest way to query Sec-Gemini but also the most inefficient. It is recommended to use the streaming API for real-time bi-directional communication. See example below.


In [6]:
# optionally add name and descriptions. If name is not set, one will be generated

# create a new session - add a name and description
session = sg.create_session()
# Test session works.
resp = session.query('What are the IP addresses of google.com?')
if resp:
    print(resp.text())
else:
    print('generate failed')


The IPv4 addresses (A records) for google.com are: 142.250.125.113, 142.250.125.100, 142.250.125.138, 142.250.125.102, 142.250.125.139, and 142.250.125.101.

The IPv6 addresses (AAAA records) for google.com are: 2607:f8b0:4001:c2f::65, 2607:f8b0:4001:c2f::8b, 2607:f8b0:4001:c2f::66, and 2607:f8b0:4001:c2f::8a.
 

 ## Sec-Gemini execution flow
 | State               | Actor         | Message Type       | Content                                                                                              |
|---------------------|---------------|--------------------|------------------------------------------------------------------------------------------------------|
| State.CALLING_TOOL  | orchestrator  | MessageType.INFO   | Calling Transfer To Agent                                                                            |
| State.CALLING_TOOL  | orchestrator  | MessageType.DEBUG  | {"id": "adk-b33cfdbb-4258-43ed-bf51-5d0ee7fa448f", "args": {"agent_name": "network_agent"}, "name":  

## accessing session data

The SDK take care of auto refreshing the session data at access time. So you can access the latest data by simply accessing the session object properties. See example below on how to display the session messages.


In [22]:
# visualize the session messages history is a simple as a for loop
for msg in session.messages:
    print(f"{msg.message_type}: {msg.get_content()}")

MessageType.QUERY: What are the IP addresses of google.com?
MessageType.INFO: Calling Transfer To Agent
MessageType.INFO: Executing network_agent
MessageType.INFO: Calling Lookup Dns Record
MessageType.INFO: Calling Lookup Dns Record
MessageType.INFO: Processing Lookup Dns Record result
MessageType.INFO: Processing Lookup Dns Record result
MessageType.RESULT: The IPv4 address for google.com is 142.251.32.46. The IPv6 address for google.com is 2607:f8b0:4005:802::200e.



## Listing available sessions

In [23]:
sg.list_sessions()

                                                                   Sessions                                                                   
╭──────────────────────────────────┬─────────────┬─────────────────┬───────┬────────┬──────────────────────┬──────────────────────┬──────────╮
│                                  │             │                 │       │        │                      │                      │ TTL      │
│ ID / Name                        │ Description │ State           │ #Msg  │ #Files │ Created              │ Updated              │ (sec)    │
├──────────────────────────────────┼─────────────┼─────────────────┼───────┼────────┼──────────────────────┼──────────────────────┼──────────┤
│ cece23f2e0574f0b8b5e86e33c0ac63d │             │ agent_done      │ 8     │ 0      │ 2025-05-04 00:57:22  │ 2025-05-04 00:57:22  │ 259200   │
│ fearless-dsa                     │             │                 │       │        │                      │                      │          │
╰──────────────────────────────────┴─────────────┴─────────────────┴───────┴────────┴──────────────────────┴──────────────────────┴──────────╯

In [24]:
## renamin the session
session.update(name="test_session", description="test_description", ttl=4242)
# session.send_feedback(score=1, comment="Great job!")
sg.list_sessions()

                                                                     Sessions                                                                      
╭──────────────────────────────────┬──────────────────┬─────────────────┬───────┬────────┬──────────────────────┬──────────────────────┬──────────╮
│                                  │                  │                 │       │        │                      │                      │ TTL      │
│ ID / Name                        │ Description      │ State           │ #Msg  │ #Files │ Created              │ Updated              │ (sec)    │
├──────────────────────────────────┼──────────────────┼─────────────────┼───────┼────────┼──────────────────────┼──────────────────────┼──────────┤
│ cece23f2e0574f0b8b5e86e33c0ac63d │ test_description │ agent_done      │ 8     │ 0      │ 2025-05-04 00:57:22  │ 2025-05-04 00:57:22  │ 4242     │
│ test_session                     │                  │                 │       │        │                      │                      │          │
╰──────────────────────────────────┴──────────────────┴─────────────────┴───────┴────────┴──────────────────────┴──────────────────────┴──────────╯

## Resuming a session
The SDK allows to resume a session by simply calling the `resume` method on the session object. This will automatically load the latest session data and allow you to continue the conversation from where you left off. See example below.

In [26]:
active_sessions = sg.get_sessions()  # get all active sessions
resumed_session = active_sessions[0]  # get the first active session
resumed_session.messages
# print(active_sessions)

[Message(id='9ff929c0312b', parent_id='3713', turn='2aa7f345a328', group='83bd18de3a5c41e8a2c2e036d633434d', actor='user', role=<Role.USER: 'user'>, timestamp=1746320242, message_type=<MessageType.QUERY: 'query'>, message_sub_type=None, state=<State.START: 'start'>, content='What are the IP addresses of google.com?', mime_type=<MimeType.TEXT: 'text/plain'>, status_code=200, status_message='OK', usage=Usage(prompt_tokens=0, generated_tokens=0, total_tokens=0) prompt_tokens=0, generated_tokens=0, total_tokens=0)),
 Message(id='6a8cd1a3fb49', parent_id='3713', turn='138ee40a47b5', group='1bb3b50ca6f2', actor='orchestrator', role=<Role.SYSTEM: 'system'>, timestamp=1746320244, message_type=<MessageType.INFO: 'info'>, message_sub_type=None, state=<State.CALLING_TOOL: 'calling_tool'>, content='Calling Transfer To Agent', mime_type=<MimeType.TEXT: 'text/plain'>, status_code=200, status_message='OK', usage=None),
 Message(id='5519205b55cd', parent_id='3713', turn='e70c4b51f924', group='c4a15eca

## Streaming API

This is the recommended way to interact with the Sec-Gemini API. It allows for real-time bi-directional communication and is more efficient than the query method. The SDK provides a simple interface to interact with the streaming API. See example below.

In [27]:
from secgemini import MessageType, MimeType
session = sg.create_session()
query = "What is the IP address of google.com?"

async for msg in session.stream(query):
    if msg.message_type == MessageType.INFO and msg.mime_type == MimeType.TEXT:
        print(f"INFO: {msg.get_content()}")
    if msg.message_type == MessageType.RESULT and msg.mime_type == MimeType.TEXT:
        # result
        print(f'\nAnswer:\n{msg.get_content()}')
        break

INFO: Calling Transfer To Agent
INFO: Executing network_agent
INFO: Calling Lookup Dns Record
INFO: Processing Lookup Dns Record result

Answer:
The IP address of google.com is 142.251.214.142.

